In [1]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from app.utils.textpreprocessing import TextPreprocessor
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

In [3]:
textprepo = TextPreprocessor()

In [4]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

In [6]:
db_faiss = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/vector_database_creation/faiss_anime_index_v3", embeddings = embedding_function)

In [7]:
import yaml

# Load API key from config.yaml
with open("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/app/config.yaml", "r") as file:
    config = yaml.safe_load(file)

api_key = config["api_key"]

In [8]:
import google.generativeai as genai
userdata = {"GOOGLE_API_KEY": api_key}
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:

def filter_tokens(metadata):
        metadata_tokens = metadata.get("tokens", [])
        metadata_studio = metadata.get("studio", [])
        metadata_producer = metadata.get("producer", [])
        metadata_licensors = metadata.get("licensors", [])
        #metadata_cf_recs = metadata.get("cf_recs", [])
        #metadata_pop_recs = metadata.get("pop_recs", [])
        metadata_genre = metadata.get("genre", [])
        return any(token in metadata_tokens for token in query_token) or metadata["score"] > 5.0 or any(token in metadata_studio for token in query_token) or any(token in metadata_producer for token in query_token) or any(token in metadata_licensors for token in query_token) or any(token in metadata_genre for token in query_token) 


In [42]:
retriever = db_faiss.as_retriever(search_kwargs={"k": 3, "filter": filter_tokens})
#retriever = db_faiss.as_retriever()

In [43]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0,google_api_key = GOOGLE_API_KEY)

In [44]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """
```Instructions
You are an anime afficinado who went deep weeb under your mothers basement who knows all about anime and japanese culture. 
Use the following pieces of context to summarize different animes at the end.
Use only the context to answer the question and provide a list of anime related to the answer.  
Recommend up to 3 anime and provide a one sentence description for each anime.
Always say "Thanks for using the anime recommender from Casual Correlations!" at the end of the answer.
```

Below is an example
```
Question: can you recommend me 5 animes from the 90

Summary: 

1. Neon Genesis Evangelion: A group of teenagers pilot giant robots to protect humanity from mysterious creatures known as Angels, in a series renowned for its complex characters and psychological themes.

2. Cowboy Bebop: This space-western series follows the adventures of a group of bounty hunters aboard the spaceship Bebop, with a jazzy soundtrack and stylish animation.

3. Dragon Ball Z: Goku and his friends defend Earth from powerful foes in this action-packed series that became a major phenomenon.


Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes from studio ghibli

1. My Neighbor Totoro (1988): Directed by Hayao Miyazaki, this heartwarming tale follows two young sisters who encounter friendly forest spirits in rural Japan.

2. Spirited Away (2001): Also directed by Hayao Miyazaki, Spirited Away tells the story of a young girl named Chihiro who becomes trapped in a mysterious and magical world, where she must work in a bathhouse for spirits to rescue her parents.

3. Princess Mononoke (1997): Directed by Hayao Miyazaki, this epic fantasy film explores the conflict between industrialization and nature, as a young prince becomes entangled in a struggle between forest gods and human settlers.


Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes similar to one piece

1. Fairy Tail: This series follows the adventures of Natsu Dragneel and his friends in the Fairy Tail guild as they take on various quests and battles in the magical land of Fiore. Like One Piece, it features a diverse cast of characters, epic battles, and a strong sense of camaraderie.

2. Naruto: Naruto follows the journey of Naruto Uzumaki, a young ninja with dreams of becoming the strongest ninja and leader of his village, the Hokage. It features a similar blend of action, humor, and heartfelt moments, as well as a focus on friendship and determination.

3. Hunter x Hunter: This series follows Gon Freecss, a young boy who aspires to become a Hunter like his father, as he embarks on various adventures and challenges in search of his father and his own identity. It shares themes of friendship, adventure, and personal growth with One Piece.


Thanks for using the anime recommender from Casual Correlations!

Question: I like the character like monkey d luffy what anime can you recommend other than one piece

1. Naruto: Naruto Uzumaki, the main character of this series, shares some similarities with Luffy. He's determined to become the strongest ninja and leader of his village, and he possesses a similar sense of optimism and loyalty to his friends.

2. Fairy Tail: Natsu Dragneel, the protagonist of Fairy Tail, is known for his adventurous spirit, boundless energy, and strong sense of loyalty to his friends, much like Luffy. The series is filled with exciting adventures and epic battles.

3. Hunter x Hunter: Gon Freecss, the main character of Hunter x Hunter, shares Luffy's sense of adventure and determination. Like Luffy, Gon is on a quest to achieve his goals and is willing to face any challenge that comes his way.

Thanks for using the anime recommender from Casual Correlations!
```

{context}

Question: {question}

Summary:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [45]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [46]:
#BlockedPromptException
#recommend popular recommendations 

In [47]:
def get_anime_recommendations(query):
    
    # Invoke the RAG chain
    streamlit_results = rag_chain.invoke(query)
    doc_results = retriever.get_relevant_documents(query)
    
    return streamlit_results, doc_results


In [48]:
# Example usage
query = 'what are good pirate anime'
query_token = textprepo.preprocess_text(query)
recommendations, doc_results = get_anime_recommendations(query)
print(recommendations)  

1. Bodacious Space Pirates: A high school girl becomes the captain of her late father's space pirate ship and embarks on adventures with her crew.

2. Space Pirate Captain Harlock: An outcast space pirate leads a crew of rebels against Earth's oppressors and a race of alien women.

3. One Piece: A young man sets out on a journey to become the Pirate King and find the greatest treasure in the world.


Thanks for using the anime recommender from Casual Correlations!


In [49]:
[(doc.metadata['anime_id'], doc.metadata['name']) for doc in doc_results]

[(8917, 'Mouretsu Pirates'),
 (17269, 'Captain Herlock'),
 (1000, 'Uchuu Kaizoku Captain Herlock')]

In [50]:
doc_results

[Document(page_content='miniskirt space pirates (ミニスカ宇宙海賊(パイレーツ), minisuka pairētsu) is a japanese light novel series about space pirates written by yūichi sasamoto and illustrated by noriyuki matsumoto, published since october 2008. an anime television series adaptation produced by satelight, under the title bodacious space pirates (モーレツ宇宙海賊(パイレーツ), mōretsu pairētsu, "fierce pirates"), aired in japan from january to june 2012. a film adaptation was released in japanese theaters in february 2014. a web manga adaptation launched in june 2012. seven seas entertainment licensed the manga series for a printed release in north america in august 2015.the anime version won the 2013 seiun award for best dramatic presentation.\n\n\n\nin the far future where space travel and colonization have become the norm, humanity has expanded its living space to the far reaches of the known galaxy. one hundred years before the beginning of the series, several colonies, eager to gain their independence, rebe

In [51]:
[doc.page_content for doc in doc_results]

['miniskirt space pirates (ミニスカ宇宙海賊(パイレーツ), minisuka pairētsu) is a japanese light novel series about space pirates written by yūichi sasamoto and illustrated by noriyuki matsumoto, published since october 2008. an anime television series adaptation produced by satelight, under the title bodacious space pirates (モーレツ宇宙海賊(パイレーツ), mōretsu pairētsu, "fierce pirates"), aired in japan from january to june 2012. a film adaptation was released in japanese theaters in february 2014. a web manga adaptation launched in june 2012. seven seas entertainment licensed the manga series for a printed release in north america in august 2015.the anime version won the 2013 seiun award for best dramatic presentation.\n\n\n\nin the far future where space travel and colonization have become the norm, humanity has expanded its living space to the far reaches of the known galaxy. one hundred years before the beginning of the series, several colonies, eager to gain their independence, rebelled against their mas